In [1]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings

plt.rc('font', size=12)
warnings.filterwarnings('ignore')

In [2]:
#Load ESCAPE training data
escapeAllData = pd.read_csv("Data/Original DataFrames/AllDataSingleValue.csv", sep=",", index_col='DEIDNUM').sort_index() #all feature dataset
escapeHemo = pd.read_csv("Data/Original DataFrames/HemoSingleValue.csv", sep=",", index_col='DEIDNUM').sort_index() #dataset with only hemodynamics
escapeLabels  = pd.read_csv("Data/Original DataFrames/Labels.csv", sep=",", index_col='DEIDNUM').sort_index() #labels for prediction classes 
escapeHemoLabels = escapeLabels[escapeLabels.index.isin(escapeHemo.index)]

#Cardiogenic Shock
cardShockHemo = pd.read_csv("Data Validation/Cardiogenic Shock/Original DataFrames/HemoCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
cardShockAllData = pd.read_csv("Data Validation/Cardiogenic Shock/Original DataFrames/AllDataCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
cardShockHemoLabels = pd.read_csv("Data Validation/Cardiogenic Shock/Original DataFrames/HemoLabelsCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
cardShockLabels = pd.read_csv("Data Validation/Cardiogenic Shock/Original DataFrames/LabelsCardiogenicShock.csv", sep=",", index_col='ID').sort_index()

#Serial Cardiac
serialHemo = pd.read_csv("Data Validation/Serial Cardiac Caths/Original DataFrames/HemoSerialCardiac.csv", sep=",", index_col='ID').sort_index()
serialAllData = pd.read_csv("Data Validation/Serial Cardiac Caths/Original DataFrames/AllDataSerialCardiac.csv", sep=",", index_col='ID').sort_index()
serialHemoLabels = pd.read_csv("Data Validation/Serial Cardiac Caths/Original DataFrames/HemoLabelsSerialCardiac.csv", sep=",", index_col='ID').sort_index()
serialLabels = pd.read_csv("Data Validation/Serial Cardiac Caths/Original DataFrames/LabelsSerialCardiac.csv", sep=",", index_col='ID').sort_index()

#HF-ACTION
hfactionAllData = pd.read_csv("Data Validation/HF-ACTION/Original DataFrames/AllDataHF-ACTION.csv", sep=",", index_col='ID').sort_index()
hfactionLabels = pd.read_csv("Data Validation/HF-ACTION/Original DataFrames/LabelsHF-ACTION.csv", sep=",", index_col='ID').sort_index()

#BEST
bestAllData = pd.read_csv("Data Validation/BEST/Original DataFrames/AllDataBEST.csv", sep=",", index_col='ID').sort_index()
bestLabels = pd.read_csv("Data Validation/BEST/Original DataFrames/LabelsBEST.csv", sep=",", index_col='ID').sort_index()

#Guide it
guideAllData = pd.read_csv("Data Validation/GUIDE-IT/Original DataFrames/AllDataGUIDE-IT.csv", sep=",", index_col='ID').sort_index()
guideLabels = pd.read_csv("Data Validation/GUIDE-IT/Original DataFrames/LabelsGUIDE-IT.csv", sep=",", index_col='ID').sort_index()


In [14]:
datasets = [escapeAllData, hfactionAllData]
labels = [escapeLabels, hfactionLabels]

bigLst = []

lst = ["n"]
for d in labels:
    lst.append(len(d))
bigLst.append(lst)

def getStats(data, labels):
    lst = []
    
    lst.append(len(labels))

    if len(data) == len(labels) * 2:
        lst.append(str(round(data.describe()['Age'][1],1)) + "±" + str(round(data.describe()['Age'][2],1)))
        gen = data['Gender'].value_counts()/2/len(labels)
        lst.append(round(gen[2.0]*100,1))
        gen = data['Race'].value_counts()/2/len(labels)
        lst.append(str(round(gen[1.0]*100,1))+"[" + str(round(gen[2.0]*100,1))+"]")
        lst.append(str(round(data.describe()['EjF'][1],1)) + "±" + str(round(data.describe()['EjF'][2],1)))
        
    else:
        lst.append(str(round(data.describe()['Age'][1],1)) + "±" + str(round(data.describe()['Age'][2],1)))
        gen = data['Gender'].value_counts()/len(labels)
        lst.append(round(gen[2.0]*100,1))
        gen = data['Race'].value_counts()/len(labels)
        lst.append(str(round(gen[1.0]*100,1))+"[" + str(round(gen[2.0]*100,1))+"]")
        lst.append(str(round(data.describe()['EjF'][1],1)) + "±" + str(round(data.describe()['EjF'][2],1)))

    lst.append(round(sum(labels['Death']) / len(labels),2))
    try:
        lst.append(round(sum(labels['Rehosp']) / len(labels),2))
    except:
        lst.append("N/A")
    try:
        lst.append(round(sum(labels['Readmission']) / len(labels),2))
    except:
        lst.append("N/A")
        

    return lst

def getStatsCath(data, labels):
    lst = []
    lst.append(len(labels))
    lst.append(str(round(data.describe()['Age'][1],1)) + "±" + str(round(data.describe()['Age'][2],1)))
    gen = data['Gender'].value_counts()/3/len(labels)
    lst.append(round(gen[2.0]*100,1))
    lst.append("N/A") #Race
    lst.append(str(round(data.describe()['EjF'][1],1)) + "±" + str(round(data.describe()['EjF'][2],1)))

    lst.append(round(sum(labels['Death']) / len(labels),2))
    lst.append("N/A")
    lst.append("N/A")
        
    return lst
    

df = pd.DataFrame(index=["n", "Age (years)", "Gender (%, female)", "Race (%, white[minority])", "EF", "Death", "Rehosp", "Readm"])
df['ESCAPE'] = getStats(escapeAllData, escapeLabels)
df['HF-ACTION'] = getStats(hfactionAllData, hfactionLabels)
df['BEST'] = getStats(bestAllData, bestLabels)
df['GUIDE-IT'] = getStats(guideAllData, guideLabels)
df['UVA Cardiogenic Shock'] = getStatsCath(cardShockAllData, cardShockLabels)
df['UVA Serial Cath'] = getStatsCath(serialAllData, serialLabels)


# df = pd.DataFrame(bigLst, columns=['idx','ESCAPE', 'HF-ACTION']).set_index("idx")
df

,ESCAPE,HF-ACTION,BEST,GUIDE-IT,UVA Cardiogenic Shock,UVA Serial Cath
n,433,2130,2707,894,364,183
Age (years),56.1±13.9,58.6±12.7,60.2±12.3,61.5±13.9,59.4±18.5,60.6±15.1
"Gender (%, female)",25.9,28.1,21.9,68,35.2,43.2
"Race (%, white[minority])",59.6[40.4],62.1[37.9],70.0[30.0],54.8[45.2],N/A,N/A
EF,19.3±6.6,25.4±7.4,23.0±7.3,24.1±8.2,31.7±17.4,31.3±18.0
Death,0.27,0.16,0.32,0.17,0.57,0.42
Rehosp,0.57,0.12,0.63,0.41,N/A,N/A
Readm,0.18,0.03,N/A,N/A,N/A,N/A


In [9]:
serialLabels

,Death
ID,
Z103426,0
Z103697,0
Z1050590,0
Z1058186,1
Z1059364,0
...,...
Z93479,1
Z93572,1
Z937913,0


In [14]:
def getStats(data, labels):
    if len(data) == len(labels) * 2:
        print("Age:", str(round(data.describe()['Age'][1],1)) + "[" + str(round(data.describe()['Age'][2],1)) + "]")
        gen = data['Gender'].value_counts()/2/len(labels)
        print("Gender", round(gen[2.0],3))
        gen = data['Race'].value_counts()/2/len(labels)
        print("Race", str(round(gen[1.0],3))+"[" + str(round(gen[2.0],3))+"]")
    else:
        print("Age:", str(round(data.describe()['Age'][1],1)) + "[" + str(round(data.describe()['Age'][2],1)) + "]")
        gen = data['Gender'].value_counts()/len(labels)
        print("Gender", round(gen[2.0],3))
        gen = data['Race'].value_counts()/len(labels)
        print("Race", str(round(gen[1.0],3))+"[" + str(round(gen[2.0],3))+"]")
    
getStats(escapeAllData, escapeLabels)

Age: 56.1[13.9]
Gender 0.259
Race 0.596[0.404]


In [21]:
getStats(escapeHemo, escapeLabels)

Age: 56.8[13.5]
Gender 0.254
Race 0.582[0.268]


In [15]:
getStats(hfactionAllData, hfactionLabels)

Age: 58.6[12.7]
Gender 0.281
Race 0.621[0.379]


In [16]:
getStats(bestAllData, bestLabels)

Age: 60.2[12.3]
Gender 0.219
Race 0.7[0.3]


In [17]:
getStats(guideAllData, guideLabels)

Age: 61.5[13.9]
Gender 0.68
Race 0.548[0.452]


In [18]:
def getStatsCard(data, labels):
    print("Age:", str(round(data.describe()['Age'][1],1)) + "[" + str(round(data.describe()['Age'][2],1)) + "]")
    gen = data['Gender'].value_counts()/3/len(labels)
    print("Gender", round(gen[2.0],3))



In [19]:
getStatsCard(cardShockAllData, cardShockAllData)

Age: 59.4[18.5]
Gender 0.117


In [20]:
getStatsCard(serialAllData, serialLabels)

Age: 60.6[15.1]
Gender 0.432
